In [ ]:
#홍채인식 알고리즘(Daugman's Algorithm) 
import cv2
import numpy as np
from scipy.ndimage import rotate

def preprocess_image(image_path):
    # 이미지를 그레이스케일로 로드
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Error: Unable to load image at {image_path}")
        return None
    # 이미지 크기를 조정
    image = cv2.resize(image, (640, 480))
    # 히스토그램 평활화를 통해 대비를 향상
    image = cv2.equalizeHist(image)
    return image

def segment_iris(image):
    # 이미지가 비어 있는지 확인
    if image is None:
        return None
    # HoughCircles를 사용하여 홍채를 찾습니다
    circles = cv2.HoughCircles(image, cv2.HOUGH_GRADIENT, 1, 20,
                               param1=50, param2=30, minRadius=30, maxRadius=80)
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            center_x, center_y, radius = i[0], i[1], i[2]
            return center_x, center_y, radius
    return None

def normalize_iris(image, center_x, center_y, radius):
    # 홍채 이미지를 원통 좌표계로 변환
    polar_array = cv2.warpPolar(image, (256, 64), (center_x, center_y), radius, cv2.WARP_POLAR_LINEAR)
    return polar_array

def gabor_filter(image):
    gabor_result = np.zeros_like(image, dtype=np.float32)
    for theta in range(0, 180, 10):  # 각도 간격을 줄임
        theta = theta * np.pi / 180
        kernel = cv2.getGaborKernel((31, 31), 4.0, theta, 8.0, 0.5, 0, ktype=cv2.CV_32F)  # 커널 크기 및 파라미터 조정
        filtered = cv2.filter2D(image, cv2.CV_8UC3, kernel)
        np.maximum(gabor_result, filtered, gabor_result)
    return gabor_result

def extract_features(image):
    # Gabor 필터를 사용하여 특징을 추출
    gabor_result = gabor_filter(image)
    features = gabor_result.flatten()
    return features

def compare_features(features1, features2):
    # 유클리드 거리로 특징을 비교
    distance = np.linalg.norm(features1 - features2)
    threshold = 500  # 임계값을 낮춰 민감도 높이기 -> iris 1 -> 6900 iris2 -> 6400 iris3 -> 6600 iris4 -> 7100 iris5 -> 6500
    return distance < threshold

def main(image1_path, image2_path):
    # 첫 번째 이미지 전처리
    image1 = preprocess_image(image1_path)
    # 두 번째 이미지 전처리
    image2 = preprocess_image(image2_path)

    if image1 is None or image2 is None:
        print("Error: One or both images could not be loaded")
        return

    # 첫 번째 이미지에서 홍채 세분화
    iris1 = segment_iris(image1)
    # 두 번째 이미지에서 홍채 세분화
    iris2 = segment_iris(image2)

    if iris1 is None or iris2 is None:
        print("Iris not detected")
        return

    center_x1, center_y1, radius1 = iris1
    center_x2, center_y2, radius2 = iris2

    # 첫 번째 이미지 노멀라이제이션
    normalized_iris1 = normalize_iris(image1, center_x1, center_y1, radius1)
    # 두 번째 이미지 노멀라이제이션
    normalized_iris2 = normalize_iris(image2, center_x2, center_y2, radius2)

    # 첫 번째 이미지에서 특징 추출
    features1 = extract_features(normalized_iris1)
    # 두 번째 이미지에서 특징 추출
    features2 = extract_features(normalized_iris2)

    # 특징 비교
    if compare_features(features1, features2):
        print("success")
    else:
        print("fail")

if __name__ == "__main__":
    image1_path = 'C:/Users/User/Desktop/programs/1234.jpg'
    image2_path = 'C:/Users/User/Desktop/programs/123455.jpg'

    main(image1_path, image2_path)

In [ ]:
#실험코드....
import cv2
import numpy as np
import pywt

def preprocess_image(image_path):
    # 이미지를 그레이스케일로 로드
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Error: Unable to load image at {image_path}")
        return None
    # 이미지 크기 조정 및 히스토그램 평활화
    image = cv2.resize(image, (640, 480))
    image = cv2.equalizeHist(image)
    return image

def is_blurred(image, mean_threshold, stddev_threshold):
    # 이미지의 평균 및 표준편차 계산
    mean, stddev = cv2.meanStdDev(image)
    mean = mean[0][0]
    stddev = stddev[0][0]
    
    is_mean_blurred = mean < mean_threshold
    is_stddev_blurred = stddev < stddev_threshold

    return is_mean_blurred or is_stddev_blurred

def segment_iris(image):
    # 이미지가 비어 있는지 확인
    if image is None:
        return None
    # Canny Edge Detection을 사용하여 엣지를 검출
    edges = cv2.Canny(image, 100, 200)
    # HoughCircles를 사용하여 홍채를 찾습니다
    circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, 1, 20,
                               param1=50, param2=30, minRadius=30, maxRadius=80)
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            center_x, center_y, radius = i[0], i[1], i[2]
            return center_x, center_y, radius
    return None

def normalize_iris(image, center_x, center_y, radius):
    # 홍채 이미지를 원통 좌표계로 변환
    polar_array = cv2.warpPolar(image, (256, 64), (center_x, center_y), radius, cv2.WARP_POLAR_LINEAR)
    return polar_array

def extract_wavelet_features(image):
    # Haar 웨이블릿 변환을 사용하여 특징을 추출
    coeffs = pywt.wavedec2(image, 'haar', level=3)
    features = []
    for coeff in coeffs:
        if isinstance(coeff, tuple):
            for subband in coeff:
                features.append(np.mean(subband))
                features.append(np.std(subband))
        else:
            features.append(np.mean(coeff))
            features.append(np.std(coeff))
    return np.array(features)

def compare_features(features1, features2):
    # 유클리드 거리로 특징을 비교
    distance = np.linalg.norm(features1 - features2)
    threshold = 500  # 임계값 설정
    return distance < threshold

def main(image1_path, image2_path, mean_threshold=100, stddev_threshold=50):
    # 첫 번째 이미지 전처리
    image1 = preprocess_image(image1_path)
    # 두 번째 이미지 전처리
    image2 = preprocess_image(image2_path)

    if image1 is None or image2 is None:
        print("Error: One or both images could not be loaded")
        return

    # 첫 번째 이미지에서 홍채 세분화
    iris1 = segment_iris(image1)
    # 두 번째 이미지에서 홍채 세분화
    iris2 = segment_iris(image2)

    if iris1 is None or iris2 is None:
        print("Iris not detected")
        return

    center_x1, center_y1, radius1 = iris1
    center_x2, center_y2, radius2 = iris2

    # 첫 번째 이미지에서 홍채 영역 잘라내기
    iris_region1 = image1[center_y1 - radius1:center_y1 + radius1, center_x1 - radius1:center_x1 + radius1]
    # 두 번째 이미지에서 홍채 영역 잘라내기
    iris_region2 = image2[center_y2 - radius2:center_y2 + radius2, center_x2 - radius2:center_x2 + radius2]

    # 홍채 영역이 블러된 경우 확인
    if is_blurred(iris_region1, mean_threshold, stddev_threshold):
        print("Error: First image's iris region is blurred")
        return
    if is_blurred(iris_region2, mean_threshold, stddev_threshold):
        print("Error: Second image's iris region is blurred")
        return

    # 첫 번째 이미지 노멀라이제이션
    normalized_iris1 = normalize_iris(image1, center_x1, center_y1, radius1)
    # 두 번째 이미지 노멀라이제이션
    normalized_iris2 = normalize_iris(image2, center_x2, center_y2, radius2)

    # 첫 번째 이미지에서 웨이블릿 특징 추출
    features1 = extract_wavelet_features(normalized_iris1)
    # 두 번째 이미지에서 웨이블릿 특징 추출
    features2 = extract_wavelet_features(normalized_iris2)

    # 특징 비교
    if compare_features(features1, features2):
        print("success")
    else:
        print("fail")

if __name__ == "__main__":
    image1_path = 'C:/Users/User/Desktop/programs/iris4.jpg'
    image2_path = 'C:/Users/User/Desktop/programs/blurred_iris_image.jpg'

    # 민감도 조절을 위해 mean_threshold 및 stddev_threshold 값을 설정
    mean_threshold = 45  # 필요에 따라 값을 조정
    stddev_threshold = 10  # 필요에 따라 값을 조정

    main(image1_path, image2_path, mean_threshold, stddev_threshold)
